# Zara Alert
How Can I check my desired item from zara.com rugularly thorugh apps rather than manually? This idea inspired me to write the code below to automate this process. At the end, I'm able to receive an email asap when an desired item is back in stock
## components:
- gmail sending(through gmail api):
send email to me when it's back in stock
- web scraping(bs4):
scrape the website and check stock every 5 minutes

## 1. [Gmail api](https://developers.google.com/gmail/api/quickstart/python):
How to send email in Python with a google account?
### 1.1 Config
as mentioned [here](https://developers.google.com/gmail/api/quickstart/python), firstly, you need Turn on the Gmail API and DOWNLOAD CLIENT CONFIGURATION and save the file `credentials.json` to your working directory. 

Also you need to install the packages below:
```
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```
### 1.2 Luanch service


In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)

Note: 

- `SCOPES` url above is permissions you request users to authorize for your app. Different settings can be found [here](https://developers.google.com/gmail/api/auth/scopes)

- At the first time you run the codes, you will be directed to your broswer to login and confirm access you give to this app.

### 1.3 [Create and send email](https://developers.google.com/gmail/api/guides/sending)

In [ ]:
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    message=message.as_string()
    # return {'raw': base64.urlsafe_b64encode(message)} # python 2
    return {'raw': base64.urlsafe_b64encode(message.encode("utf-8")).decode("utf-8")}# python 3

def send_message(service, user_id, message):
    message = (service.users()
    .messages()
    .send(userId=user_id, 
    body=message)
    .execute())
    print ('Message Id: %s' % message['id'])
    return message

Note: functions above are defined by goole. The only difference is `base64.urlsafe_b64encode(message.encode("utf-8")).decode("utf-8")` in python3 is necessary due to different deconding in Python 2 and 3'

Now we can easily create and send email by functions above

In [ ]:
my_email=
message=create_message(my_email,my_email,'zara','there is stock')
send_message(service,'me',message) 

`'me'` is a special input represents the email by which you download the `credentials.json`

## 2. [Scraping](https://www.reddit.com/r/Python/comments/3yexp5/scraping_gbp_price_info_from_zaracom_with/)
The item I desire and wait for is [this](https://www.zara.com/ca/en/double-faced-faux-leather-jacket-p06318351.html). Let's scrape the page first.

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome"}
rootUrl = "https://www.zara.com/ca/en/double-faced-faux-leather-jacket-p06318351.html"

session = requests.Session()

def get_page():
    req = session.get(rootUrl, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    return soup.find("div", {"class": "product-size-selector__size-list-wrapper"})

Atfer inspecting the website, I found an `div` element whose class is `product-size-selector__size-list-wrapper`, listing all sizes and their stocks. For each size `li` element in this list, it `class` attribute must includes `product-size-selector__size-list-item--out-of-stock`, hence this is the key to determine when to send email.

In [ ]:
def check_item(size=0): # 0 means the smallest size in order
    size_selector=get_page()
    smallest_size=size_selector.find_all('li')[0]
    return 'product-size-selector__size-list-item--out-of-stock' in smallest_size['class']

## App
now we can easily populate the app now ~

In [ ]:
count=0
while check_item():
    count+=1
    print(count,'-th check')
    time.sleep(300) # Sleep for 300 seconds
    
message=create_message(my_email,my_email,'zara','there is stock')
send_message(service,'me',message)